In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents.initialize import initialize_agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

No module named 'phonopy'
No module named 'phonopy'


In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool()]
tools += [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, Wikipedia, and a python 
    REPL, which you can use to execute python code. If you get an error, debug 
    your code and try again. Only use the output of your code to answer the 
    question. Ask user to clarify their queries if needed. Please note that you 
    don't have direct control over MP but through multiple assistant agents to 
    help you. You need to provide complete context in the input for assistants to 
    do their job.
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:

while True:
    response = agent_executor.invoke(
        {
            "input": input("Question:"),
        }
    )



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: piezoelectric materials
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: piezoelectric materials
Page: Piezoelectricity
Summary: Piezoelectricity (, US: ) is the electric charge that accumulates in certain solid materials—such as crystals, certain ceramics, and biological matter such as bone, DNA, and various proteins—in response to applied mechanical stress. The word piezoelectricity means electricity resulting from pressure and latent heat. It is derived from Ancient Greek  πιέζω (piézō) 'to squeeze or press', and  ἤλεκτρον (ḗlektron) 'amber' (an ancient source of static electricity).  The German form of the word (Piezoelektricität) was coined in 1881 by the German physicist Wilhelm Gottlieb Hankel; the English word was coined in 1883.The piezoelectric effect results from the linear electromechanical interaction between the mechanical and electrical state

/home/cyrus/miniconda3/envs/llamp/lib/python3.12/site-packages/mp_api/client/core/client.py:993: UserWarning: The server does not support the request made to https://api.materialsproject.org/materials/summary/?_fields=material_id%2Cformula_pretty%2Ccomposition%2Cnsites%2Csymmetry&formula=Pb%28Zr&_limit=500. This may be due to an outdated mp-api package, or a problem with the query.
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.12/site-packages/mp_api/client/core/client.py:993: UserWarning: The server does not support the request made to https://api.materialsproject.org/materials/summary/?_fields=material_id%2Cformula_pretty%2Ccomposition%2Cnsites%2Csymmetry&formula=Ti%29O3&_limit=500. This may be due to an outdated mp-api package, or a problem with the query.
  warnings.warn(


Error on search_materials_summary__get: REST query returned with error status code 400 on URL https://api.materialsproject.org/materials/summary/?_fields=material_id%2Cformula_pretty%2Ccomposition%2Cnsites%2Csymmetry&formula=Ti%29O3&_limit=500 with message:
Problem processing one or more provided formulas.. Please provide more detailed reqeust arguments or try smaller request by specifying 'limit' in request.Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Pb(Zr,Ti)O3",
    "fields": "material_id,formula_pretty,composition,nsites,symmetry",
    "limit": 5
  }
}
```Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Pb(Zr,Ti)O3",
    "fields": "material_id,formula_pretty,composition,nsites,symmetry",
    "limit": 5
  }
}
```

/home/cyrus/miniconda3/envs/llamp/lib/python3.12/site-packages/mp_api/client/core/client.py:993: UserWarning: The server does not support the request made to https://api.materialsproject.org/materials/summary/?_fields=material_id%2Cformula_pretty%2Ccomposition%2Cnsites%2Csymmetry&formula=Ti%29O3&_limit=500. This may be due to an outdated mp-api package, or a problem with the query.
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.12/site-packages/mp_api/client/core/client.py:993: UserWarning: The server does not support the request made to https://api.materialsproject.org/materials/summary/?_fields=material_id%2Cformula_pretty%2Ccomposition%2Cnsites%2Csymmetry&formula=Pb%28Zr&_limit=500. This may be due to an outdated mp-api package, or a problem with the query.
  warnings.warn(


Error on search_materials_summary__get: REST query returned with error status code 400 on URL https://api.materialsproject.org/materials/summary/?_fields=material_id%2Cformula_pretty%2Ccomposition%2Cnsites%2Csymmetry&formula=Ti%29O3&_limit=500 with message:
Problem processing one or more provided formulas.. Please provide more detailed reqeust arguments or try smaller request by specifying 'limit' in request.Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "PbZrO3,PbTiO3",
    "fields": "material_id,formula_pretty,composition,nsites,symmetry",
    "limit": 5
  }
}
```Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "PbZrO3,PbTiO3",
    "fields": "material_id,formula_pretty,composition,nsites,symmetry",
    "limit": 5
  }
}
```

...

Retrieving PiezoelectricDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'material_id': 'mp-20459', 'last_updated': '2023-03-11T03:21:02.084000'}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The material with ID mp-20459 was last updated on 2023-03-11T03:21:02.084000."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The material with ID mp-20459 was last updated on 2023-03-11T03:21:02.084000."
}
```

> Finished chain.
{'input': 'mp-20459', 'output': 'The material with ID mp-20459 was last updated on 2023-03-11T03:21:02.084000.'}It seems I made an error by not requesting the piezoelectric tensor data specifically for the material with ID mp-20459. I will correct this and try again.

Thought: Do I need to use a tool? Yes
Action: MPPiezoelectricExpert
Action Input: mp-20459 piezoelectric tensorIt seems I made an error by not requesting the piezoelectric tensor data specifically for the material with ID mp-20459. I will correct this and try again.

Thought: Do I need to use a tool? Yes
Action: MPPiezoelectricExpert
Action Inp

Retrieving PiezoelectricDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'material_id': 'mp-20459', 'e_ij_max': 3.6330572166427553}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The piezoelectric tensor maximum component (e_ij_max) for the material with ID mp-20459 is 3.633 C/m^2."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The piezoelectric tensor maximum component (e_ij_max) for the material with ID mp-20459 is 3.633 C/m^2."
}
```

> Finished chain.
{'input': 'mp-20459 piezoelectric tensor', 'output': 'The piezoelectric tensor maximum component (e_ij_max) for the material with ID mp-20459 is 3.633 C/m^2.'}Final Answer: The piezoelectric tensor maximum component (e_ij_max) for the material with ID mp-20459 (lead zirconate titanate, Pb(Zr,Ti)O3) is 3.633 C/m^2.Final Answer: The piezoelectric tensor maximum component (e_ij_max) for the material with ID mp-20459 (lead zirconate titanate, Pb(Zr,Ti)O3) is 3.633 C/m^2.

> Finished chain.


> Entering new AgentExecutor chain...
It seems like there is missing information in 

KeyboardInterrupt: 